# Format data for STM

Need to integrate the computed IMCEs for the survey respondents and the text responses and recode some key variables with interpretable names for modelling.

Variables to be re-mapped: 
| Survey Question | Variable Description                                                                                                                  |
|-----------------|------------------------------------------------------------------------------------------------------------------------------------------|
| ResponseId      | ID of survey response                                                                                    |
| Q12.6_corrected      | Field with the open text response data (in this case, manually spell corrected and filtered)
| Q3.1            | Gender                                                                                                                                   |
| Q3.3            | Age (categorical)                                                                                                                        |
| Q3.5            | Income      (categorical)                                                                                                                             |
| Q3.7            | Citizenship                                                                                                                              |
| Q3.11           | Experience of housing discrimination                                                                                                     |
| Q3.16           | City district                                                                                                                            |
| Q4.1            | City neighborhood                                                                                                                        |
| Q5.1            | Professional situation                                                                                                                   |
| Q5.5            | Education                                                                                                                                |
| Q6.1            | Pre-frame acceptance of densification projects                                                                                           |
| Q6.5_3          | Fear of being forced out of your home                                                                                                     |
| Q6.5_6          | Seen landlord pressure                                                                                                                    |
| Q6.5_9          | Seen changes in the community                                                                                                                  |
| Q6.5_10         | Seen renovations                                                                                                                          |
| Q6.11_1         | Neighborhood connection                                                                                                                   |
| Q6.15_5         | Respect for rules                                                                                                                         |
| Q6.15_6         | Neighborhood crime                                                                                                                        |
| Q7.5            | Post-frame acceptance of densification projects                                                                                           |
| Q7.6            | Densification projects' effect on rent (perception)                                                                                                   |
| Q7.7            | Densification projects' effect on social mix (perception)                                                                                              |
| Q14.1_1         | In terms of political issues people often talk about liberal (left) and conservative (right). Where would you position yourself?         | 
| Q14.3           | On a scale of 1-10 please rate how environmental pollution affects your personal health and safety where 1 affects you the least and 10 affects you the most.|
| Q14.9           | Climate concern                                                                                                                          |

## Read in data, modify variable names and encodings and merge IMCE information

### Read in data, modify variable names and encodings 
Import package for data manipulation, specify the columns to use (aka the survey variables to include), and the ways they will be encoded more substantively and more interpretably for the STM modelling.


In [1]:
import pandas as pd

In [2]:
#columns to include from survey
subset_col = 'Q12.6_corrected' # this is the text column
cols_to_include = [subset_col, 'frame', 'minority', 'linguistic_minority', 'religious_minority', 'social_class_minority', 
                   'sexual_orientation_minority', 'ethnic_minority', 'national_minority', 'other_minority', 
                   'disabled_minority', 'Q3.1', 'Q3.3', 'Q3.5', 'Q3.7', 'Q5.5', 'Q14.1_1', 
                   'Q3.11', 'Q3.16', 'Q4.1', 'Q5.1', 'Q6.5_3', 'Q6.5_6', 'Q6.5_9', 'Q6.5_10', 'Q6.11_1', 'Q6.15_5', 'Q6.15_6', 
                   'Q14.3', 'Q14.9', 'age', 'city', 'country', 'Q6.1', 'Q7.5', 'Q7.6', 'Q7.7']

#create dictionary for mapping the column names to a readable/interpretable form based on the survey question
substantive_colnames_map = {'ResponseId': 'id', subset_col : 'doc', 
                       'Q3.1': 'gender', 'Q3.3': 'age_numerical', 'Q3.5': 'income', 'Q3.7': 'citizenship', 'Q3.11': 'exp_housing_discrim',
                       'Q3.16': 'city_district', 'Q4.1': 'city_neighborhood', 'Q5.1': 'prof_situation', 'Q5.5': 'education', 
                       'Q6.5_3': 'fear_forced_out', 'Q6.5_6': 'seen_landlord_pressure', 'Q6.5_9': 'changes_in_community', 
                       'Q6.5_10': 'seen_renovations', 'Q6.11_1': 'neighborhood_connection', 'Q6.15_5': 'respect_rules', 
                       'Q6.15_6': 'neighbhorhood_crime', 'Q14.1_1': 'political_left_right', 
                       'Q14.3': 'env_pollution_impact', 'Q14.9': 'climate_concern', 'Q6.1': 'preframe_densifproj_acceptance', 
                       'Q7.5': 'postframe_densifproj_acceptance', 'Q7.6': 'densifproj_on_rent', 'Q7.7': 'densifproj_on_social'}

#create scales to map the likert style variables to a numerical scale (columns specified below)
mapping_likert = {'do not agree at all': -2, 'do not agree': -1, 'neutral': 0, 'agree': 1, 'fully agree': 2}
mapping_likert_neighborhood_connection = {'not connected at all': -2, 'not very connected': -1, 'neutral': 0, 'somewhat connected': 1, 'strongly connected': 2}
mapping_likert_climate_concern = {'Not concerned': -2, 'Rather not concerned': -1, 'Neither': 0, 'Quite concerned': 1, 'Very concerned': 2}
mapping_likert_acceptance = {'always reject': -3, 'reject': -2, 'mostly reject': -1, 'undecided': 0, 'mostly accept': 1, 'accept': 2, 'always accept': 3}
mapping_likert_inc_dec = {'heavily decrease': -3, 'decrease': -2, 'mostly decrease': -1, 'no change': 0, 'mostly increase': 1, 'increase': 2, 'heavily increase': 3}

#create scale to map the political affiliation: 
mapping_political = {0.0: 'liberal', 1.0: 'liberal', 2.0: 'liberal', 3.0: 'moderate', 4.0: 'moderate', 5.0: 'conservative',  6.0: 'conservative', 7.0: 'conservative'}
mapping_political_v2 = {0.0: 'very liberal', 1.0: 'liberal', 2.0: 'liberal', 3.0: 'moderate', 
                        4.0: 'moderate', 5.0: 'conservative',  6.0: 'conservative', 7.0: 'very conservative'}

#create scale to map the age (categorical)
mapping_age = {'55+': 5, '45-54': 4, '35-44': 3, '25-34': 2, '18-24': 1}

#specify columns of each variable type
likert_cols = ['fear_forced_out', 'seen_landlord_pressure', 'changes_in_community', 'seen_renovations', 'respect_rules', 
               'neighbhorhood_crime']

dummy_cols = ['frame', 'gender', 'citizenship', 'education', 'exp_housing_discrim', 'city_district', 'city_neighborhood', 
              'prof_situation', 'city', 'country']

In [3]:
#import data
survey_data_filtered_file = '../data/spur_survey_response_filtered_df1.txt'
survey_df = pd.read_csv(survey_data_filtered_file, sep='\t')

df = survey_df[['ResponseId'] + cols_to_include]
df = df.rename(columns=substantive_colnames_map)

#remap some columns
for col in likert_cols: 
    df[col] = df[col].map(mapping_likert)
df['climate_concern'] = df['climate_concern'].map(mapping_likert_climate_concern)
df['neighborhood_connection'] = df['neighborhood_connection'].map(mapping_likert_neighborhood_connection)
df['age'] = df['age'].map(mapping_age)
df['preframe_densifproj_acceptance'] = df['preframe_densifproj_acceptance'].map(mapping_likert_acceptance)
df['postframe_densifproj_acceptance'] = df['postframe_densifproj_acceptance'].map(mapping_likert_acceptance)
df['densifproj_on_rent'] = df['densifproj_on_rent'].map(mapping_likert_inc_dec)
df['densifproj_on_social'] = df['densifproj_on_social'].map(mapping_likert_inc_dec)
df['politics'] = df['political_left_right'].map(mapping_political)
df['politics_w_extremes'] = df['political_left_right'].map(mapping_political_v2)

#if want to just add gender as a dummy var:
df_gender_dummy = pd.get_dummies(df, columns=['gender']) 

#NOTE: we wont use this in our analysis, but we can also make a version of the dataframe with more dummy variables: 
df_w_dummies = pd.get_dummies(df, columns=dummy_cols)

print(df.info(), df_w_dummies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4886 entries, 0 to 4885
Data columns (total 40 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   id                               4886 non-null   object 
 1   doc                              4886 non-null   object 
 2   frame                            4886 non-null   object 
 3   minority                         4886 non-null   float64
 4   linguistic_minority              4886 non-null   float64
 5   religious_minority               4886 non-null   float64
 6   social_class_minority            4886 non-null   float64
 7   sexual_orientation_minority      4886 non-null   float64
 8   ethnic_minority                  4886 non-null   float64
 9   national_minority                4886 non-null   float64
 10  other_minority                   4886 non-null   float64
 11  disabled_minority                4886 non-null   float64
 12  gender              

### Merge IMCE information
Merge the IMCE information into the dataframe as well and visualize an example of text reponses from respondents that score highly for a particular imce example

In [4]:
## Read in IMCE data
imce_filepath = '../data/imce_df_final_corrected.csv'
imce_df = pd.read_csv(imce_filepath)
print(imce_df.columns)

print('Dr. Wicki computed IMCEs for a total of %d participants across the surveys' % len(imce_df))
imce_df.head()

#add imce data to survey df
survey_df = df.merge(imce_df, how='left', left_on='id', right_on='Group.1')

#use dataframe with recoded gender 
survey_df = df_gender_dummy.merge(imce_df, how='left', left_on='id', right_on='Group.1')

Index(['Group.1', 'imce_RentContr', 'imce_InclZon', 'imce_Partic',
       'imce_nonProf', 'imce_dens', 'imce_mixUse', 'imce_ClimNeutr'],
      dtype='object')
Dr. Wicki computed IMCEs for a total of 12611 participants across the surveys


In [5]:
incl_Zone_highest_imces = list(survey_df[['id', 'doc', 'imce_InclZon']].sort_values(by='imce_InclZon', ascending=False)['doc'].iloc[:25])
incl_Zone_highest_imces

['Public participation and rent control ',
 'Carbon neutral are good ideas. Prefer use of mix of buildings e.g not just apartments. ',
 'people need to have some stability on living conditions',
 ' the inclusion of low-income or affordable housing was most important to me, no preference on who the investor was, slight preference for rent control or rent capping',
 "I'm more in favour of a higher proportion of affordable or social housing, and some involvement in the development.",
 "There needs to be more inclusion of small businesses and craft stores because that's the best way to support",
 "I'm more in favour of local involvement, environmental concerns and mixed planning that creates an actual neighbourhood rather than just a dormitory commuter zone.",
 "WE NEED TO BE MORE RESPONSIBLE ESPECIALLY WITH CLIMATE CONTROL. IT'S BAD ENOUGH THE CITY IS EXPANDING BUT IT'S TIME TO TAKE RESPONSIBILITY FOR IT",
 'The percentage of affordable housing',
 'Affordable housing is crucial',
 'One of

### Save output file
Take a look at the final dataframe and export it with the corrected IMCEs and recoded variables (including gender dummy and recoded politics columns) to a file for stm

In [6]:
print(list(survey_df.columns))
survey_df['densifproj_on_rent'].value_counts()
survey_df.head()
#survey_df['age'].value_counts()
#survey_df.to_csv('../datasets/spur_survey_response_filtered_df1_for_stm_corrected_imces_v2.csv', index=False, sep=',')

['id', 'doc', 'frame', 'minority', 'linguistic_minority', 'religious_minority', 'social_class_minority', 'sexual_orientation_minority', 'ethnic_minority', 'national_minority', 'other_minority', 'disabled_minority', 'age_numerical', 'income', 'citizenship', 'education', 'political_left_right', 'exp_housing_discrim', 'city_district', 'city_neighborhood', 'prof_situation', 'fear_forced_out', 'seen_landlord_pressure', 'changes_in_community', 'seen_renovations', 'neighborhood_connection', 'respect_rules', 'neighbhorhood_crime', 'env_pollution_impact', 'climate_concern', 'age', 'city', 'country', 'preframe_densifproj_acceptance', 'postframe_densifproj_acceptance', 'densifproj_on_rent', 'densifproj_on_social', 'politics', 'politics_w_extremes', 'gender_Female', 'gender_Male', 'gender_Non-binary', 'gender_Prefer not to say', 'Group.1', 'imce_RentContr', 'imce_InclZon', 'imce_Partic', 'imce_nonProf', 'imce_dens', 'imce_mixUse', 'imce_ClimNeutr']


,id,doc,frame,minority,linguistic_minority,religious_minority,social_class_minority,sexual_orientation_minority,ethnic_minority,national_minority,...,gender_Non-binary,gender_Prefer not to say,Group.1,imce_RentContr,imce_InclZon,imce_Partic,imce_nonProf,imce_dens,imce_mixUse,imce_ClimNeutr
0,R_10PI5FKTTlId8Ec,I'm not yo I'm not talking to you,other district,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0,0,R_10PI5FKTTlId8Ec,0.189348,-0.681112,-0.315282,0.348762,0.047983,0.218712,0.122512
1,R_1gduid1fizdQ4d8,It is the way it is,other district,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,R_1gduid1fizdQ4d8,-0.529477,-0.256714,-0.600379,-0.025870,0.223711,0.246441,-0.216726
2,R_23WCgmyuPAy3b1G,Because it would be more fair to people that d...,other district,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0,0,R_23WCgmyuPAy3b1G,-0.334249,-0.042235,-0.441566,-0.192129,-0.115328,0.341073,0.206765
3,R_1pLEMXp5iJbankG,I choose proposal 2 because you offered me bet...,other district,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0,0,R_1pLEMXp5iJbankG,0.539111,-0.322129,0.630386,0.229208,0.592402,0.040068,-0.475606
4,R_2OYEXKWutkzQOmS,Because I felt like it wouldn't be good enough...,other district,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,R_2OYEXKWutkzQOmS,-0.539888,-0.077108,-0.452229,0.365875,0.093102,0.178249,0.614162
